In [ ]:
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train_scaled"
test_dir = image_path / "test_scaled"

train_dir, test_dir
output_dir = 'output'

In [ ]:

import torch
torch.cuda.empty_cache()
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [3]:
import matplotlib.pyplot as plt
import torch

from torch import nn
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s

weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights=weights).to(device)

In [4]:
auto_transforms = weights.transforms()

In [ ]:
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=auto_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=auto_transforms)


In [ ]:

class_names = train_data.classes
class_names

['Combat',
 'DestroyedBuildings',
 'Fire',
 'Humanitarian Aid and rehabilitation',
 'Military vehicles and weapons']

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=16,shuffle=False)

In [ ]:
# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)
# Recreate the classifier layer and seed it to the target device

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
).to(device)

In [ ]:
from torch import nn
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

In [ ]:
torch.cuda.set_device(0)

In [ ]:
next(model.parameters()).is_cuda

True

In [ ]:
# Set the random seeds
try:
    from going_modular.going_modular import engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !move pytorch-deep-learning\going_modular .
    !rmdir /s /q pytorch-deep-learning
    from going_modular.going_modular import engine

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=15,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.4064 | train_acc: 0.5062 | test_loss: 1.0437 | test_acc: 0.8625
Epoch: 2 | train_loss: 0.7421 | train_acc: 0.8781 | test_loss: 0.5772 | test_acc: 0.9375
Epoch: 3 | train_loss: 0.5056 | train_acc: 0.9781 | test_loss: 0.5388 | test_acc: 0.9250
Epoch: 4 | train_loss: 0.4776 | train_acc: 0.9750 | test_loss: 0.5137 | test_acc: 0.9625
Epoch: 5 | train_loss: 0.4496 | train_acc: 0.9938 | test_loss: 0.4985 | test_acc: 0.9625
Epoch: 6 | train_loss: 0.4573 | train_acc: 0.9812 | test_loss: 0.5094 | test_acc: 0.9625
Epoch: 7 | train_loss: 0.4293 | train_acc: 1.0000 | test_loss: 0.5029 | test_acc: 0.9625
Epoch: 8 | train_loss: 0.4222 | train_acc: 1.0000 | test_loss: 0.4977 | test_acc: 0.9750
Epoch: 9 | train_loss: 0.4306 | train_acc: 0.9969 | test_loss: 0.5107 | test_acc: 0.9500
Epoch: 10 | train_loss: 0.4349 | train_acc: 0.9969 | test_loss: 0.4982 | test_acc: 0.9500
Epoch: 11 | train_loss: 0.4220 | train_acc: 1.0000 | test_loss: 0.5042 | test_acc: 0.9625
Epoch: 12 | train_l

In [ ]:
torch.save(model.state_dict(), 'weights.tf')